In [1]:
import os
%pwd

'd:\\Kidney-Disease-Classification\\notebooks'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from KidneyDisease.constants import *
from KidneyDisease.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import zipfile
import gdown
from KidneyDisease import logger
from KidneyDisease.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        """
        Downloading the data from the url.
        """
        try:
            dataset_url = self.config.source_URL 
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading the data from {dataset_url} into file {zip_download_dir}")
            
            ## See trails.ipynb for info
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded the data from {dataset_url}")
            
        except Exception as e:
            raise e
        
    def extract_zipfile(self):
        """
        Extracts the zip file into the directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

    
except Exception as e:
    raise e

[2023-10-09 11:43:12,190: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-09 11:43:12,190: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 11:43:12,192: INFO: common: created directory at: artifacts]
[2023-10-09 11:43:12,193: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-09 11:43:12,193: INFO: 2765119257: Downloading the data from https://drive.google.com/file/d/1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR/view?usp=sharing into file artifacts/data_ingestion/Data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR
From (redirected): https://drive.google.com/uc?/export=download&id=1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR&confirm=t&uuid=c8346278-a502-4ea1-88bf-2fdbe6dbe05e
To: d:\Kidney-Disease-Classification\artifacts\data_ingestion\Data.zip
100%|██████████| 1.63G/1.63G [01:43<00:00, 15.7MB/s]

[2023-10-09 11:44:57,709: INFO: 2765119257: Downloaded the data from https://drive.google.com/file/d/1R7bzHew6_jzoDZ2vFbQmoUxT9jiGHmZR/view?usp=sharing]
